# Exercise - Part 1

### Here we will do the cleaning of the data during the scraping operation itself. We will remove the postal codes with no boroughs assigned and also do all the required formattings.

In [17]:
from bs4 import BeautifulSoup
import requests
import csv
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')
tbody = table.find('tbody')

csv_file = open('canada_ps.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostalCode', 'Borough', 'Neighborhood'])

for tr in tbody.find_all('tr')[1:]:
    post = tr.find_all('td')[0].text.strip()
    borough = tr.find_all('td')[1].text.strip()
    neighborhood = tr.find_all('td')[2].text.strip()
    if borough != 'Not assigned':
        if (len(neighborhood.strip()) == 0):
            neighborhood = borough
        try:
            neighborhood = neighborhood.replace(" /",",")
        except:
            pass
        csv_writer.writerow([post, borough, neighborhood])
        
csv_file.close()

Now, we have the formatted data stored in the csv file. Let us now import the data as a pandas data frame

In [18]:
import pandas as pd
df = pd.read_csv('canada_ps.csv')

In [24]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [20]:
df.shape

(103, 3)

In [ ]:
import geocoder 


lat_lng_coords = None

while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]